# 1 The complete python code used in 3.3 of the paper is as follows:

In [ ]:
import numpy as np
import random
import math
import matplotlib.pyplot as plt
import time

def rastrigin(x, A=10):
    n = len(x)
    return A * n + sum(x_i**2 - A * np.cos(2 * np.pi * x_i) for x_i in x)

def sphere(x):
    return sum(x_i**2 for x_i in x)

def rosenbrock(x):
    return sum(100 * (x_i_next - x_i**2)**2 + (1 - x_i)**2 for x_i, x_i_next in zip(x[:-1], x[1:]))

def ackley(x):
    part1 = -0.2 * np.sqrt(0.5 * sum(x_i**2 for x_i in x))
    part2 = sum(np.cos(2 * np.pi * x_i) for x_i in x) / len(x)
    return -20 * np.exp(part1) - np.exp(part2) + 20 + np.e

def griewank(x):
    part1 = 1 / 4000 * sum(x_i**2 for x_i in x)
    part2 = np.prod([np.cos(x_i / np.sqrt(i + 1)) for i, x_i in enumerate(x)])
    return part1 - part2 + 1

def levi(x):
    term1 = np.sin(3 * np.pi * x[0])**2
    term2 = sum((x_i - 1)**2 * (1 + 10 * np.sin(np.pi * x_i + 1)**2) for x_i in x[1:])
    term3 = (x[-1] - 1)**2 * (1 + np.sin(2 * np.pi * x[-1])**2)
    return term1 + term2 + term3

def schwefel(x):
    return 418.9829 * len(x) - sum(x_i * np.sin(np.sqrt(abs(x_i))) for x_i in x)

def rastrigin_2(x, A=5.12):
    n = len(x)
    return A * n + sum(x_i**2 - A * np.cos(2 * np.pi * x_i) for x_i in x)

def zakharov(x):
    term1 = sum(x_i**2 for x_i in x)
    term2 = sum(0.5 * (i + 1) * x_i for i, x_i in enumerate(x))
    term3 = sum(0.5 * (i + 1) * x_i for i, x_i in enumerate(x))**2
    return term1 + term2 + term3

def michalewicz(x):
    m = 10
    return -sum(np.sin(x_i) * np.sin((i + 1) * x_i**2 / np.pi)**(2 * m) for i, x_i in enumerate(x))

functions = [rastrigin, sphere, rosenbrock, ackley, griewank, levi, schwefel, rastrigin_2, zakharov, michalewicz]

# 1. Advanced Ceramic Process-Inspired Metaheuristic Optimization (ACP-MO)
class ACPMO:
    def __init__(self, population_size, dimension, max_iter, temperature=1.0):
        self.population_size = population_size
        self.dimension = dimension
        self.max_iter = max_iter
        self.temperature = temperature
        self.population = np.random.uniform(-5.12, 5.12, (population_size, dimension))
        self.best_solution = None
        self.best_fitness = float('inf')

    def update_temperature(self, iteration):
        return self.temperature * (1 - iteration / self.max_iter)

    def forming_operator(self, x):
        return x + np.random.normal(0, 0.1, size=x.shape)

    def reverse_design_healing(self, x):
        return x + np.random.normal(0, 0.05, size=x.shape)  # Repair function

    def optimize(self, func):
        for iteration in range(self.max_iter):
            new_population = []
            for i in range(self.population_size):
                x = self.population[i]
                # Forming phase (like base shape forming)
                new_solution = self.forming_operator(x)
                # Firing phase (evaluation)
                fitness = func(new_solution)
                if fitness < self.best_fitness:
                    self.best_fitness = fitness
                    self.best_solution = new_solution
                new_population.append(new_solution)
            self.population = np.array(new_population)
            self.temperature = self.update_temperature(iteration)
        return self.best_solution, self.best_fitness

# 2. Genetic Algorithm (GA)
class GA:
    def __init__(self, population_size, dimension, max_iter, crossover_prob=0.8, mutation_prob=0.2):
        self.population_size = population_size
        self.dimension = dimension
        self.max_iter = max_iter
        self.crossover_prob = crossover_prob
        self.mutation_prob = mutation_prob
        self.population = np.random.uniform(-5.12, 5.12, (population_size, dimension))
        self.best_solution = None
        self.best_fitness = float('inf')

    def crossover(self, parent1, parent2):
        if np.random.rand() < self.crossover_prob:
            crossover_point = np.random.randint(1, self.dimension)
            return np.concatenate([parent1[:crossover_point], parent2[crossover_point:]])
        return parent1

    def mutation(self, solution):
        if np.random.rand() < self.mutation_prob:
            mutation_point = np.random.randint(0, self.dimension)
            solution[mutation_point] = np.random.uniform(-5.12, 5.12)
        return solution

    def optimize(self, func):
        for iteration in range(self.max_iter):
            new_population = []
            for i in range(self.population_size // 2):
                parent1, parent2 = self.population[np.random.choice(self.population_size, 2, replace=False)]
                offspring1 = self.crossover(parent1, parent2)
                offspring2 = self.crossover(parent2, parent1)
                new_population.append(self.mutation(offspring1))
                new_population.append(self.mutation(offspring2))

            self.population = np.array(new_population)
            for solution in self.population:
                fitness = func(solution)
                if fitness < self.best_fitness:
                    self.best_fitness = fitness
                    self.best_solution = solution
        return self.best_solution, self.best_fitness

# 3. Particle Swarm Optimization (PSO)
class PSO:
    def __init__(self, population_size, dimension, max_iter, w=0.5, c1=1.5, c2=1.5):
        self.population_size = population_size
        self.dimension = dimension
        self.max_iter = max_iter
        self.w = w  # Inertia weight
        self.c1 = c1  # Cognitive coefficient
        self.c2 = c2  # Social coefficient
        self.population = np.random.uniform(-5.12, 5.12, (population_size, dimension))
        self.velocity = np.random.uniform(-1, 1, (population_size, dimension))
        self.best_solution = np.copy(self.population)
        self.best_fitness = np.array([float('inf')] * population_size)
        self.global_best_solution = None
        self.global_best_fitness = float('inf')

    def optimize(self, func):
        for iteration in range(self.max_iter):
            for i in range(self.population_size):
                fitness = func(self.population[i])
                if fitness < self.best_fitness[i]:
                    self.best_fitness[i] = fitness
                    self.best_solution[i] = self.population[i]

                if fitness < self.global_best_fitness:
                    self.global_best_fitness = fitness
                    self.global_best_solution = self.population[i]

            for i in range(self.population_size):
                self.velocity[i] = (self.w * self.velocity[i] +
                                    self.c1 * np.random.rand(self.dimension) * (self.best_solution[i] - self.population[i]) +
                                    self.c2 * np.random.rand(self.dimension) * (self.global_best_solution - self.population[i]))
                self.population[i] = self.population[i] + self.velocity[i]

        return self.global_best_solution, self.global_best_fitness

# 4. Differential Evolution (DE)
class DE:
    def __init__(self, population_size, dimension, max_iter, F=0.8, CR=0.9):
        self.population_size = population_size
        self.dimension = dimension
        self.max_iter = max_iter
        self.F = F  # Differential weight
        self.CR = CR  # Crossover probability
        self.population = np.random.uniform(-5.12, 5.12, (population_size, dimension))
        self.best_solution = None
        self.best_fitness = float('inf')

    def optimize(self, func):
        for iteration in range(self.max_iter):
            new_population = []
            for i in range(self.population_size):
                a, b, c = self.population[np.random.choice(self.population_size, 3, replace=False)]
                mutant = a + self.F * (b - c)
                trial = np.where(np.random.rand(self.dimension) < self.CR, mutant, self.population[i])
                fitness = func(trial)
                if fitness < self.best_fitness:
                    self.best_fitness = fitness
                    self.best_solution = trial
                new_population.append(trial)
            self.population = np.array(new_population)
        return self.best_solution, self.best_fitness

# 5. Cuckoo Search (CS)
class CS:
    def __init__(self, population_size, dimension, max_iter):
        self.population_size = population_size
        self.dimension = dimension
        self.max_iter = max_iter
        self.population = np.random.uniform(-5.12, 5.12, (population_size, dimension))
        self.best_solution = None
        self.best_fitness = float('inf')

    def optimize(self, func):
        for iteration in range(self.max_iter):
            new_population = []
            for i in range(self.population_size):
                x = self.population[i]
                new_solution = x + np.random.normal(0, 0.1, size=x.shape)
                fitness = func(new_solution)
                if fitness < self.best_fitness:
                    self.best_fitness = fitness
                    self.best_solution = new_solution
                new_population.append(new_solution)
            self.population = np.array(new_population)
        return self.best_solution, self.best_fitness

# 6. Firefly Algorithm (FA)
class FA:
    def __init__(self, population_size, dimension, max_iter, beta=1, alpha=0.2):
        self.population_size = population_size
        self.dimension = dimension
        self.max_iter = max_iter
        self.beta = beta  # Attraction
        self.alpha = alpha  # Randomness
        self.population = np.random.uniform(-5.12, 5.12, (population_size, dimension))
        self.best_solution = None
        self.best_fitness = float('inf')

    def optimize(self, func):
        for iteration in range(self.max_iter):
            new_population = []
            for i in range(self.population_size):
                x = self.population[i]
                fitness = func(x)
                if fitness < self.best_fitness:
                    self.best_fitness = fitness
                    self.best_solution = x
                # Flashing phase
                for j in range(self.population_size):
                    if fitness < func(self.population[j]):
                        x_new = x + self.beta * (self.population[j] - x) + self.alpha * np.random.normal(0, 0.1, size=x.shape)
                        new_population.append(x_new)
            self.population = np.array(new_population)
        return self.best_solution, self.best_fitness

# 7. Ant Colony Optimization (ACO) for continuous optimization (simplified)
class ACO:
    def __init__(self, population_size, dimension, max_iter, alpha=1, beta=2, evaporation_rate=0.5):
        self.population_size = population_size
        self.dimension = dimension
        self.max_iter = max_iter
        self.alpha = alpha  # pheromone importance
        self.beta = beta    # heuristic importance
        self.evaporation_rate = evaporation_rate
        self.pheromone = np.ones((population_size, dimension))
        self.best_solution = None
        self.best_fitness = float('inf')
        # search space boundaries
        self.lower_bound = -5.12
        self.upper_bound = 5.12

    def optimize(self, func):
        # initialize ants randomly
        ants = np.random.uniform(self.lower_bound, self.upper_bound, (self.population_size, self.dimension))

        for iteration in range(self.max_iter):
            fitness = np.array([func(ant) for ant in ants])
            # Update best solution
            min_idx = np.argmin(fitness)
            if fitness[min_idx] < self.best_fitness:
                self.best_fitness = fitness[min_idx]
                self.best_solution = ants[min_idx].copy()
            # Update pheromone
            self.pheromone = (1 - self.evaporation_rate) * self.pheromone
            for i in range(self.population_size):
                # better ants deposit more pheromone
                contribution = 1.0 / (fitness[i] + 1e-10)
                self.pheromone[i] += contribution
            # Construct new ants probabilistically
            for i in range(self.population_size):
                prob = (self.pheromone[i] ** self.alpha) * ((1.0 / (fitness[i] + 1e-10)) ** self.beta)
                prob = prob / np.sum(prob)
                for d in range(self.dimension):
                    if np.random.rand() < prob[d]:
                        ants[i,d] = ants[i,d] + np.random.normal(0, 0.1)
                # Keep within bounds
                ants[i] = np.clip(ants[i], self.lower_bound, self.upper_bound)
        return self.best_solution, self.best_fitness

# 8. Simulated Annealing (SA)
class SA:
    def __init__(self, population_size, dimension, max_iter, initial_temp=100, cooling_rate=0.95):
        self.population_size = population_size
        self.dimension = dimension
        self.max_iter = max_iter
        self.temp = initial_temp
        self.cooling_rate = cooling_rate
        self.lower_bound = -5.12
        self.upper_bound = 5.12
        self.best_solution = np.random.uniform(self.lower_bound, self.upper_bound, dimension)
        self.best_fitness = float('inf')

    def neighbor(self, solution):
        # Generate a neighbor by adding small noise
        new_solution = solution + np.random.normal(0, 0.5, size=self.dimension)
        new_solution = np.clip(new_solution, self.lower_bound, self.upper_bound)
        return new_solution

    def optimize(self, func):
        current_solution = self.best_solution.copy()
        current_fitness = func(current_solution)

        for iteration in range(self.max_iter):
            new_solution = self.neighbor(current_solution)
            new_fitness = func(new_solution)
            if new_fitness < current_fitness:
                current_solution, current_fitness = new_solution, new_fitness
            else:
                # Accept worse solution with some probability
                p = math.exp(-(new_fitness - current_fitness) / self.temp)
                if np.random.rand() < p:
                    current_solution, current_fitness = new_solution, new_fitness

            if current_fitness < self.best_fitness:
                self.best_fitness = current_fitness
                self.best_solution = current_solution.copy()
            # Cool down temperature
            self.temp *= self.cooling_rate
        return self.best_solution, self.best_fitness

# 9. Whale Optimization Algorithm (WOA)
class WOA:
    def __init__(self, population_size, dimension, max_iter):
        self.population_size = population_size
        self.dimension = dimension
        self.max_iter = max_iter
        self.lower_bound = -5.12
        self.upper_bound = 5.12
        self.population = np.random.uniform(self.lower_bound, self.upper_bound, (population_size, dimension))
        self.best_solution = None
        self.best_fitness = float('inf')

    def optimize(self, func):
        a = 2  # parameter linearly decreased from 2 to 0

        for t in range(self.max_iter):
            a_t = 2 - t * (2 / self.max_iter)
            for i in range(self.population_size):
                fitness = func(self.population[i])
                if fitness < self.best_fitness:
                    self.best_fitness = fitness
                    self.best_solution = self.population[i].copy()

            for i in range(self.population_size):
                r1, r2 = np.random.rand(), np.random.rand()
                A = 2 * a_t * r1 - a_t
                C = 2 * r2
                p = np.random.rand()
                if p < 0.5:
                    if abs(A) < 1:
                        D = abs(C * self.best_solution - self.population[i])
                        self.population[i] = self.best_solution - A * D
                    else:
                        rand_idx = np.random.randint(0, self.population_size)
                        X_rand = self.population[rand_idx]
                        D = abs(C * X_rand - self.population[i])
                        self.population[i] = X_rand - A * D
                else:
                    # spiral update
                    D_prime = abs(self.best_solution - self.population[i])
                    b = 1  # constant for logarithmic spiral
                    l = np.random.uniform(-1,1)
                    self.population[i] = D_prime * np.exp(b * l) * np.cos(2 * np.pi * l) + self.best_solution
                # Keep within bounds
                self.population[i] = np.clip(self.population[i], self.lower_bound, self.upper_bound)

        return self.best_solution, self.best_fitness

# Coati Optimization Algorithm (COA)

class Coati:
    def __init__(self, population_size, dimension, max_iter):
        self.population_size = population_size
        self.dimension = dimension
        self.max_iter = max_iter
        self.lower_bound = -5.12
        self.upper_bound = 5.12
        self.population = np.random.uniform(self.lower_bound, self.upper_bound, (population_size, dimension))
        self.best_solution = None
        self.best_fitness = float('inf')

    def optimize(self, func):
        for t in range(self.max_iter):
            fitness = np.array([func(ind) for ind in self.population])
            min_idx = np.argmin(fitness)
            if fitness[min_idx] < self.best_fitness:
                self.best_fitness = fitness[min_idx]
                self.best_solution = self.population[min_idx].copy()

            for i in range(self.population_size):
                r1 = np.random.rand()
                if r1 < 0.5:
                    step = r1 * (self.best_solution - self.population[i])
                else:
                    rand_idx = np.random.randint(self.population_size)
                    step = r1 * (self.population[rand_idx] - self.population[i])
                self.population[i] += step
                self.population[i] = np.clip(self.population[i], self.lower_bound, self.upper_bound)
        return self.best_solution, self.best_fitness

# 11. Aquila Optimization Algorithm (AOA)  
class Aquila:
    def __init__(self, population_size, dimension, max_iter):
        self.population_size = population_size
        self.dimension = dimension
        self.max_iter = max_iter
        self.lower_bound = -5.12
        self.upper_bound = 5.12
        self.population = np.random.uniform(self.lower_bound, self.upper_bound, (population_size, dimension))
        self.best_solution = None
        self.best_fitness = float('inf')

    def optimize(self, func):
        for t in range(self.max_iter):
            fitness = np.array([func(ind) for ind in self.population])
            min_idx = np.argmin(fitness)
            if fitness[min_idx] < self.best_fitness:
                self.best_fitness = fitness[min_idx]
                self.best_solution = self.population[min_idx].copy()

            for i in range(self.population_size):
                F = np.random.rand()
                if t / self.max_iter < 0.5:
                    if F < 0.5:
                        # Exploration
                        self.population[i] += F * np.random.normal(size=self.dimension)
                    else:
                        # Broad exploration
                        self.population[i] += np.random.uniform(-1, 1, size=self.dimension)
                else:
                    # Exploitation
                    self.population[i] = self.best_solution + F * (np.random.rand(self.dimension) - 0.5)
                self.population[i] = np.clip(self.population[i], self.lower_bound, self.upper_bound)
        return self.best_solution, self.best_fitness

# 12. Prairie Dog Optimization (PDO)

class PDO:
    def __init__(self, population_size, dimension, max_iter):
        self.population_size = population_size
        self.dimension = dimension
        self.max_iter = max_iter
        self.lower_bound = -5.12
        self.upper_bound = 5.12
        self.population = np.random.uniform(self.lower_bound, self.upper_bound, (population_size, dimension))
        self.best_solution = None
        self.best_fitness = float('inf')

    def optimize(self, func):
        for t in range(self.max_iter):
            fitness = np.array([func(ind) for ind in self.population])
            min_idx = np.argmin(fitness)
            if fitness[min_idx] < self.best_fitness:
                self.best_fitness = fitness[min_idx]
                self.best_solution = self.population[min_idx].copy()

            for i in range(self.population_size):
                social_factor = np.random.rand()
                step = social_factor * (self.best_solution - self.population[i]) + \
                       np.random.normal(0, 0.1, self.dimension)
                self.population[i] += step
                self.population[i] = np.clip(self.population[i], self.lower_bound, self.upper_bound)
        return self.best_solution, self.best_fitness

    

from collections import defaultdict
import numpy as np
import time

def compare_algorithms():
    population_size = 30
    dimension = 10
    max_iter = 100
    runs = 100  # number of repetitions

    fitness_data = defaultdict(lambda: defaultdict(list))  # function → algorithm → fitness list

    for func in functions:
        for algorithm in ["ACPMO", "GA", "PSO", "DE", "CS", "FA", "ACO", "SA", "WOA", "COA", "AOA", "PDO"]:
            for _ in range(runs):
                start_time = time.time()

                if algorithm == "ACPMO":
                    optimizer = ACPMO(population_size, dimension, max_iter)
                elif algorithm == "GA":
                    optimizer = GA(population_size, dimension, max_iter)
                elif algorithm == "PSO":
                    optimizer = PSO(population_size, dimension, max_iter)
                elif algorithm == "DE":
                    optimizer = DE(population_size, dimension, max_iter)
                elif algorithm == "CS":
                    optimizer = CS(population_size, dimension, max_iter)
                elif algorithm == "FA":
                    optimizer = FA(population_size, dimension, max_iter)
                elif algorithm == "ACO":
                    optimizer = ACO(population_size, dimension, max_iter)
                elif algorithm == "SA":
                    optimizer = SA(population_size, dimension, max_iter)
                elif algorithm == "WOA":
                    optimizer = WOA(population_size, dimension, max_iter)
                elif algorithm == "COA":
                    optimizer = Coati(population_size, dimension, max_iter)
                elif algorithm == "AOA":
                    optimizer = Aquila(population_size, dimension, max_iter)
                elif algorithm == "PDO":
                    optimizer = PDO(population_size, dimension, max_iter)

                best_solution, best_fitness = optimizer.optimize(func)
                fitness_data[func.__name__][algorithm].append(best_fitness)

    return fitness_data

from scipy.stats import friedmanchisquare, wilcoxon
import scipy.stats as stats

def analyze_results(fitness_data):
    for func_name, alg_results in fitness_data.items():
        print(f"\n=== Function: {func_name} ===")
        algorithms = list(alg_results.keys())
        data_matrix = np.array([alg_results[algo] for algo in algorithms])  # shape: (n_algorithms, n_runs)

        # --- Descriptive statistics ---
        print("\nMean ± Std Dev:")
        for algo in algorithms:
            values = np.array(alg_results[algo])
            mean = np.mean(values)
            std = np.std(values)
            print(f"{algo}: {mean:.4f} ± {std:.4f}")

        # Optional: 95% Confidence Interval
        print("\nMean ± 95% Confidence Interval:")
        for algo in algorithms:
            values = np.array(alg_results[algo])
            mean = np.mean(values)
            sem = stats.sem(values)  # standard error of the mean
            ci = 1.96 * sem
            print(f"{algo}: {mean:.4f} ± {ci:.4f}")

        # --- Friedman test (non-parametric test for multiple related samples) ---
        stat, p_value = friedmanchisquare(*data_matrix)
        print(f"\nFriedman test: statistic = {stat:.4f}, p-value = {p_value:.4e}")
        if p_value < 0.05:
            print("→ Significant differences exist among the algorithms (p < 0.05)")

        # --- Pairwise Wilcoxon test (optional) ---
        print("\nWilcoxon signed-rank test (vs baseline algorithm: {})".format(algorithms[0]))
        baseline = np.array(alg_results[algorithms[0]])
        for algo in algorithms[1:]:
            comparison = np.array(alg_results[algo])
            try:
                stat, p = wilcoxon(baseline, comparison)
                print(f"{algorithms[0]} vs {algo}: p-value = {p:.4e}" + (" → significant" if p < 0.05 else ""))
            except ValueError:
                print(f"{algorithms[0]} vs {algo}: test could not be performed (e.g., all differences are zero)")

fitness_data = compare_algorithms()
analyze_results(fitness_data)

import matplotlib.pyplot as plt

def plot_results(fitness_data):
    for func_name, alg_results in fitness_data.items():
        algorithms = list(alg_results.keys())
        means = [np.mean(alg_results[algo]) for algo in algorithms]
        stds = [np.std(alg_results[algo]) for algo in algorithms]

        plt.figure(figsize=(12, 6))
        plt.bar(algorithms, means, yerr=stds, capsize=5, alpha=0.7, color='skyblue')
        plt.ylabel('Best Fitness (lower is better)')
        plt.xlabel('Algorithm')
        plt.title(f'Algorithm Performance on {func_name}')
        plt.xticks(rotation=45)
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()
        plt.show()

fitness_data = compare_algorithms()
analyze_results(fitness_data)
plot_results(fitness_data)


# 2 The complete python code used in 3.4 of the paper is as follows:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# === Test Function (Sphere) ===
def sphere(x):
    return np.sum(x ** 2)

# === Advanced Ceramic Process-Inspired Metaheuristic Optimization (ACP-MO) ===
class ACPMO:
    def __init__(self, population_size, dimension, max_iter, temperature=1.0, repair_noise_std=0.05):
        self.population_size = population_size
        self.dimension = dimension
        self.max_iter = max_iter
        self.temperature = temperature
        self.repair_noise_std = repair_noise_std
        self.population = np.random.uniform(-5.12, 5.12, (population_size, dimension))
        self.best_solution = None
        self.best_fitness = float('inf')

    def update_temperature(self, iteration):
        return self.temperature * (1 - iteration / self.max_iter)

    def forming_operator(self, x):
        return x + np.random.normal(0, 0.1, size=x.shape)

    def reverse_design_healing(self, x):
        return x + np.random.normal(0, self.repair_noise_std, size=x.shape)

    def optimize(self, func):
        for iteration in range(self.max_iter):
            new_population = []
            for i in range(self.population_size):
                x = self.population[i]
                new_solution = self.forming_operator(x)
                new_solution = self.reverse_design_healing(new_solution)
                fitness = func(new_solution)
                if fitness < self.best_fitness:
                    self.best_fitness = fitness
                    self.best_solution = new_solution
                new_population.append(new_solution)
            self.population = np.array(new_population)
            self.temperature = self.update_temperature(iteration)
        return self.best_solution, self.best_fitness

# === Sensitivity Analysis Function ===
def sensitivity_analysis(param_name, values, runs=10):
    results = []
    print(f"\n=== Sensitivity Analysis: {param_name} ===")
    for val in values:
        fitness_list = []
        for _ in range(runs):
            if param_name == 'population_size':
                optimizer = ACPMO(population_size=val, dimension=30, max_iter=100)
            elif param_name == 'temperature':
                optimizer = ACPMO(population_size=30, dimension=30, max_iter=100, temperature=val)
            elif param_name == 'repair_noise_std':
                optimizer = ACPMO(population_size=30, dimension=30, max_iter=100, repair_noise_std=val)
            else:
                raise ValueError("Invalid parameter")
            _, best_fit = optimizer.optimize(sphere)
            fitness_list.append(best_fit)
        mean = np.mean(fitness_list)
        std = np.std(fitness_list)
        results.append((val, mean, std))
        # === Text output ===
        print(f"Value: {val}, Mean Fitness: {mean:.6f}, Std: {std:.6f}")

    # === Plotting the sensitivity curve ===
    x_vals = [x[0] for x in results]
    y_vals = [x[1] for x in results]
    y_errs = [x[2] for x in results]

    plt.figure(figsize=(8, 5))
    plt.errorbar(x_vals, y_vals, yerr=y_errs, fmt='-o', capsize=5, color='blue')
    plt.xlabel(param_name)
    plt.ylabel('Best Fitness (lower is better)')
    plt.title(f'Sensitivity Analysis of {param_name}')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# === Run all sensitivity experiments ===
if __name__ == "__main__":
    print("Running sensitivity analysis on population size...")
    sensitivity_analysis('population_size', [10, 20, 30, 40, 50])

    print("Running sensitivity analysis on temperature...")
    sensitivity_analysis('temperature', [0.1, 0.5, 1.0, 2.0, 5.0])

    print("Running sensitivity analysis on repair noise std...")
    sensitivity_analysis('repair_noise_std', [0.01, 0.05, 0.1, 0.2, 0.5])


# 3 The complete python code used in 3.5 of the paper is as follows:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# === Test Function (Sphere) ===
def sphere(x):
    return np.sum(x ** 2)

# === Advanced Ceramic Process-Inspired Metaheuristic Optimization (ACP-MO) ===
class ACPMO:
    def __init__(self, population_size, dimension, max_iter, temperature=1.0,
                 use_forming=True, use_firing=True, use_healing=True,
                 repair_noise_std=0.05):
        """
        Initialize optimizer with options to enable/disable forming, firing, healing phases.
        """
        self.population_size = population_size
        self.dimension = dimension
        self.max_iter = max_iter
        self.temperature = temperature
        self.repair_noise_std = repair_noise_std

        # Enable or disable each component for ablation study
        self.use_forming = use_forming
        self.use_firing = use_firing
        self.use_healing = use_healing

        self.population = np.random.uniform(-5.12, 5.12, (population_size, dimension))
        self.best_solution = None
        self.best_fitness = float('inf')

    def update_temperature(self, iteration):
        """
        Temperature decay schedule.
        """
        return self.temperature * (1 - iteration / self.max_iter)

    def forming_operator(self, x):
        """
        Forming phase: add Gaussian noise perturbation.
        """
        return x + np.random.normal(0, 0.1, size=x.shape)

    def reverse_design_healing(self, x):
        """
        Healing phase: add smaller Gaussian noise to repair solution.
        """
        return x + np.random.normal(0, self.repair_noise_std, size=x.shape)

    def optimize(self, func):
        """
        Main optimization loop.
        Supports enabling/disabling forming, firing, healing phases.
        """
        for iteration in range(self.max_iter):
            new_population = []
            for i in range(self.population_size):
                x = self.population[i]

                # Apply forming operator if enabled
                if self.use_forming:
                    new_solution = self.forming_operator(x)
                else:
                    new_solution = np.copy(x)

                # Apply healing operator if enabled
                if self.use_healing:
                    new_solution = self.reverse_design_healing(new_solution)

                # Evaluate fitness (firing phase)
                if self.use_firing:
                    fitness = func(new_solution)
                else:
                    # If firing disabled, skip evaluation and keep old solution
                    fitness = func(x)
                    new_solution = x

                # Update best found solution
                if fitness < self.best_fitness:
                    self.best_fitness = fitness
                    self.best_solution = new_solution

                new_population.append(new_solution)

            self.population = np.array(new_population)
            self.temperature = self.update_temperature(iteration)

        return self.best_solution, self.best_fitness

# === Ablation Study to test component effectiveness ===
def ablation_study():
    """
    Test ACPMO with all combinations of forming, firing, healing enabled/disabled.
    Run each configuration multiple times and report mean and std dev of best fitness.
    """
    configs = [
        {'use_forming': True,  'use_firing': True,  'use_healing': True},
        {'use_forming': False, 'use_firing': True,  'use_healing': True},
        {'use_forming': True,  'use_firing': False, 'use_healing': True},
        {'use_forming': True,  'use_firing': True,  'use_healing': False},
        {'use_forming': False, 'use_firing': False, 'use_healing': True},
        {'use_forming': False, 'use_firing': True,  'use_healing': False},
        {'use_forming': True,  'use_firing': False, 'use_healing': False},
        {'use_forming': False, 'use_firing': False, 'use_healing': False},
    ]

    runs_per_config = 10
    population_size = 30
    dimension = 30
    max_iter = 100

    print("=== Ablation Study on ACP-MO Components ===")
    print(f"{'Forming':^10} | {'Firing':^10} | {'Healing':^10} | {'Mean Fitness':^15} | {'Std Dev':^10}")
    print("-"*65)

    results = []

    for config in configs:
        fitnesses = []
        for _ in range(runs_per_config):
            optimizer = ACPMO(
                population_size=population_size,
                dimension=dimension,
                max_iter=max_iter,
                temperature=1.0,
                use_forming=config['use_forming'],
                use_firing=config['use_firing'],
                use_healing=config['use_healing'],
                repair_noise_std=0.05,
            )
            _, best_fit = optimizer.optimize(sphere)
            fitnesses.append(best_fit)
        mean_fit = np.mean(fitnesses)
        std_fit = np.std(fitnesses)
        results.append((config['use_forming'], config['use_firing'], config['use_healing'], mean_fit, std_fit))
        print(f"{str(config['use_forming']):^10} | {str(config['use_firing']):^10} | {str(config['use_healing']):^10} | {mean_fit:^15.6f} | {std_fit:^10.6f}")

    # Plot results for visualization
    labels = [f"F:{f} Fg:{fi} H:{h}" for f, fi, h, _, _ in results]
    means = [r[3] for r in results]
    stds = [r[4] for r in results]

    plt.figure(figsize=(10,6))
    plt.bar(range(len(labels)), means, yerr=stds, capsize=5)
    plt.xticks(range(len(labels)), labels, rotation=45)
    plt.ylabel('Mean Best Fitness (lower better)')
    plt.title('Ablation Study on ACP-MO Components')
    plt.tight_layout()
    plt.grid(axis='y')
    plt.show()

if __name__ == "__main__":
    ablation_study()


# 4 The complete python code used in 3.6 of the paper is as follows:

In [ ]:
import numpy as np

# === Real-World Engineering Test Problem: Pressure Vessel Design ===
# Objective: Minimize cost while satisfying engineering constraints

def pressure_vessel_design(x):
    x = np.clip(x, [1, 1, 10, 10], [100, 100, 200, 200])
    x1, x2, x3, x4 = x
    cost = (
        0.6224 * x1 * x3 * x4 +
        1.7781 * x2 * x3**2 +
        3.1661 * x1**2 * x4 +
        19.84 * x1**2 * x3
    )
    # Constraints (penalty-based)
    g1 = -x1 + 0.0193 * x3
    g2 = -x2 + 0.00954 * x3
    g3 = -np.pi * x3**2 * x4 - (4/3) * np.pi * x3**3 + 1296000
    g4 = x4 - 240

    penalty = 0
    for g in [g1, g2, g3, g4]:
        if g > 0:
            penalty += 1e6 * g  # heavy penalty

    return cost + penalty

# === Advanced Ceramic Process-Inspired Metaheuristic Optimization (ACP-MO) ===
class ACPMO:
    def __init__(self, population_size, dimension, max_iter, temperature=1.0):
        self.population_size = population_size
        self.dimension = dimension
        self.max_iter = max_iter
        self.temperature = temperature
        self.population = np.random.uniform([1, 1, 10, 10], [100, 100, 200, 200], (population_size, dimension))
        self.best_solution = None
        self.best_fitness = float('inf')

    # === Temperature decay function ===
    def update_temperature(self, iteration):
        return self.temperature * (1 - iteration / self.max_iter)

    # === Forming phase: introduce variation ===
    def forming_operator(self, x):
        return x + np.random.normal(0, 0.1, size=x.shape)

    # === Healing phase: fine-tune solution ===
    def reverse_design_healing(self, x):
        return x + np.random.normal(0, 0.05, size=x.shape)

    # === Optimization loop ===
    def optimize(self, func):
        for iteration in range(self.max_iter):
            new_population = []
            for i in range(self.population_size):
                x = self.population[i]
                new_solution = self.forming_operator(x)
                new_solution = self.reverse_design_healing(new_solution)
                fitness = func(new_solution)
                if fitness < self.best_fitness:
                    self.best_fitness = fitness
                    self.best_solution = new_solution
                new_population.append(new_solution)
            self.population = np.array(new_population)
            self.temperature = self.update_temperature(iteration)
        return self.best_solution, self.best_fitness

# === Run the real-world test ===
if __name__ == "__main__":
    optimizer = ACPMO(population_size=30, dimension=4, max_iter=100)
    best_sol, best_fit = optimizer.optimize(pressure_vessel_design)

    print("=== Real-World Application: Pressure Vessel Design ===")
    print("Best Design Variables (x1, x2, x3, x4):", best_sol)
    print("Minimum Cost Found:", best_fit)
